In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 

from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

In [2]:
SEED = 7
np.random.seed(SEED)

#Data preparing 
data = pd.read_excel("Revised data.xlsx")
data.columns

Index(['Indicator Code', 'FP index ', 'LP index ', 'Vegetal Pds-FS',
       'Cereals -FS', 'Starchy Rts-FS', 'Pulses-FS', 'Fruits -FS', 'Meat-FS',
       'Fish-FS', 'Sugar & Swt-FS', 'Oils-FS ', 'Vegetables-FS ', 'Spices-FS',
       'Eggs-FS', 'Milk-FS', 'Cereals-LSF', 'Starchy Rts-LSF', 'Pulses-LSF',
       'Meat-LSF', 'Fish-LSF', 'Cereals-LS', 'Starchy-LS', 'Fruits-LS',
       'Energy use', 'Renewable energy ', 'GHGEs ', 'AG land ',
       'Agricultural water share ', 'Surface temp.', 'Overweight', 'Obese',
       'Death-NCD ', 'Life expectancy '],
      dtype='object')

In [3]:
X_data = data[[ 'FP index ', 'LP index ', 'Vegetal Pds-FS',
       'Cereals -FS', 'Starchy Rts-FS', 'Pulses-FS', 'Fruits -FS', 'Meat-FS',
       'Fish-FS', 'Sugar & Swt-FS', 'Oils-FS ', 'Vegetables-FS ', 'Spices-FS',
       'Eggs-FS', 'Milk-FS', 'Cereals-LSF', 'Starchy Rts-LSF', 'Pulses-LSF',
       'Meat-LSF', 'Fish-LSF', 'Cereals-LS', 'Starchy-LS', 'Fruits-LS',
       'Energy use', 'Renewable energy ']]
Y_data = data[['GHGEs ', 'AG land ',
       'Agricultural water share ', 'Surface temp.', 'Overweight', 'Obese','Death-NCD ', 'Life expectancy ']]    

In [4]:
Env_data = [X_data,Y_data[(['GHGEs ', 'AG land ',
       'Agricultural water share ', 'Surface temp.'])]]

Health_data = [X_data,Y_data[(['Overweight', 'Obese','Death-NCD ', 'Life expectancy '])]]



Environ_data = pd.concat(Env_data, axis =1)
Healthimp_data = pd.concat(Health_data, axis =1)

In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

X_data=Healthimp_data[[ 'FP index ', 'LP index ', 'Vegetal Pds-FS',
       'Cereals -FS', 'Starchy Rts-FS', 'Pulses-FS', 'Fruits -FS', 'Meat-FS',
       'Fish-FS', 'Sugar & Swt-FS', 'Oils-FS ', 'Vegetables-FS ', 'Spices-FS',
       'Eggs-FS', 'Milk-FS', 'Cereals-LSF', 'Starchy Rts-LSF', 'Pulses-LSF',
       'Meat-LSF', 'Fish-LSF', 'Cereals-LS', 'Starchy-LS', 'Fruits-LS',
       'Energy use', 'Renewable energy ']]
Y_data = Healthimp_data[['Overweight','Obese', 'Death-NCD ', 'Life expectancy ']]

scaler = StandardScaler(copy=False)
scaler.fit(X_data)
scaler.transform(X_data) 

scaler = StandardScaler(copy=False)
scaler.fit(X_data)
XX = scaler.transform(X_data) 

X_x = pd.DataFrame(XX, columns = ['FP index ', 'LP index ', 'Vegetal Pds-FS',
       'Cereals -FS', 'Starchy Rts-FS', 'Pulses-FS', 'Fruits -FS', 'Meat-FS',
       'Fish-FS', 'Sugar & Swt-FS', 'Oils-FS ', 'Vegetables-FS ', 'Spices-FS',
       'Eggs-FS', 'Milk-FS', 'Cereals-LSF', 'Starchy Rts-LSF', 'Pulses-LSF',
       'Meat-LSF', 'Fish-LSF', 'Cereals-LS', 'Starchy-LS', 'Fruits-LS',
       'Energy use', 'Renewable energy '], dtype = float )

x_train = X_x[:52]
x_test = X_x[52:]
y_train = Y_data[:52]
y_test = Y_data[52:]

In [6]:
# Importing all models and libraries 
from sklearn.datasets import load_linnerud
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score, KFold
from sklearn.datasets import load_linnerud
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import ElasticNet, Lasso, Ridge, LogisticRegression, LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from pyGRNN import GRNN
IGRNN = GRNN()
rounds = 20



# Adboost model

In [ ]:
grid_param_Adb = {'estimator__learning_rate':[0.001,0.01,0.1,1,2],
                   'estimator__loss':['linear'], 
                   'estimator__n_estimators':[30,40,50,60,70]
                    }
#Create arrays to store the scores
not_nested_scores_Adb  = np.zeros(rounds)
nested_scores_Adb = np.zeros(rounds)

for i in range(rounds):

   #Define both cross-validation objects (inner & outer)
   inner_cv = KFold(n_splits=5, shuffle=True, random_state=i)
   outer_cv = KFold(n_splits=3, shuffle=True, random_state=i)


   # Non-nested parameter search and scoring
   clf_Adb  = GridSearchCV(estimator=MultiOutputRegressor(AdaBoostRegressor()), param_grid=grid_param_Adb , cv=inner_cv)
   clf_Adb.fit(x_train,y_train)
   not_nested_scores_Adb [i] = clf_Adb.best_score_

   # Nested CV with parameter optimization
   nested_score_Adb = cross_val_score(clf_Adb, x_train,y_train, cv=outer_cv)
   nested_scores_Adb[i] = nested_score_Adb.mean()
   print("nested_score",nested_score_Adb)
print("R2_nested:",nested_scores_Adb.mean() )
print("R2 non_nested:",not_nested_scores_Adb.mean())
print( clf_Adb.best_params_)

nested_score [0.9728161  0.962645   0.99020709]
nested_score [0.97877864 0.97486629 0.9802228 ]
nested_score [0.98518021 0.98390498 0.97094118]
nested_score [0.98422445 0.98370321 0.98690527]


In [ ]:
plt.style.use('seaborn')
plt.tight_layout()
plt.figure(figsize=(7,5))
sns.set(font_scale=1.2)
outer_scores_line, = plt.plot(not_nested_scores_Adb, color='orange', linewidth=3.5)
nested_line, = plt.plot(nested_scores_Adb, color='steelblue', linewidth=3.5)
# plt.ylim(0.95, 0.995)
plt.ylabel("Cross-validation score", fontsize="15")
plt.xlabel("Number of trials", fontsize="15")
plt.legend([outer_scores_line, nested_line],
          ["Non-Nested CV_AdaBoost", "Nested CV_AdaBoost"],
          bbox_to_anchor=(0, .18, 1.02, 0), fontsize = 13)
# plt.title("Non-Nested vs Nested Cross-Validation",
#          x=.5, y=1.1, fontsize="15")
plt.savefig('Cross-val_Adaboost.png', dpi = 1000, bbox_inches='tight')

# GRNN model 

In [ ]:
grid_param_IGRNN = { 'estimator__kernel':["RBF"],
                 'estimator__sigma' : [0.01, 0.1,0.4, 1 , 2,4,10],
                 'estimator__calibration' : ['None'], 'estimator__n_restarts_optimizer':[1,0, 0.01, 0.001]
                    }





#Create arrays to store the scores
not_nested_scores_IGRNN = np.zeros(rounds)
nested_scores_IGRNN = np.zeros(rounds)

for i in range(rounds):

   #Define both cross-validation objects (inner & outer)
   inner_cv = KFold(n_splits=5, shuffle=True, random_state=i)
   outer_cv = KFold(n_splits=3, shuffle=True, random_state=i)


   # Non-nested parameter search and scoring
   clf_IGRNN  = GridSearchCV(estimator=MultiOutputRegressor(IGRNN), param_grid=grid_param_IGRNN , cv=inner_cv)
   clf_IGRNN.fit(x_train,y_train)
   not_nested_scores_IGRNN [i] = clf_IGRNN.best_score_

   # Nested CV with parameter optimization
   nested_score_IGRNN = cross_val_score(clf_IGRNN, x_train,y_train, cv=outer_cv)
   nested_scores_IGRNN[i] = nested_score_IGRNN.mean()
   print("nested_score",nested_score_IGRNN )
print("R2 nested:",nested_scores_IGRNN.mean() )
print("R2 not_nested:",not_nested_scores_IGRNN.mean())
print( clf_IGRNN.best_params_)

In [ ]:
plt.style.use('seaborn')
sns.set(font_scale=1.2)
plt.tight_layout()
plt.figure(figsize=(7,5))
outer_scores_line, = plt.plot(not_nested_scores_IGRNN, color='orange',  linewidth=3.5)
nested_line, = plt.plot(nested_scores_IGRNN, color='steelblue',  linewidth=3.5)
plt.ylabel("Cross-validation score", fontsize="15")
plt.xlabel("Number of trials", fontsize="15")
plt.ylim(0.97, 0.99)
plt.legend([outer_scores_line, nested_line],
          ["Non-Nested CV_IGRNN", "Nested CV_IGRNN"],
          bbox_to_anchor=(0, .2, 1, 0), fontsize = 13)
# plt.title("Non-Nested vs Nested Cross-Validation",
#          x=.5, y=1.1, fontsize="15")
# plt.savefig('Cross_val IGRNN.png', dpi = 1000, bbox_inches='tight')

# KNN model 

In [ ]:
grid_param_knn = {'estimator__leaf_size':[1,5, 10 ,15],
                   'estimator__n_neighbors':[2,3,4,5,6], 
                    'estimator__p':[2,3,4,5]
                    }
#Create arrays to store the scores
not_nested_scores_knn  = np.zeros(rounds)
nested_scores_knn = np.zeros(rounds)

for i in range(rounds):

   #Define both cross-validation objects (inner & outer)
   inner_cv = KFold(n_splits=5, shuffle=True, random_state=i)
   outer_cv = KFold(n_splits=3, shuffle=True, random_state=i)


   # Non-nested parameter search and scoring
   clf_knn  = GridSearchCV(estimator=MultiOutputRegressor(KNeighborsRegressor()), param_grid=grid_param_knn , cv=inner_cv)
   clf_knn.fit(x_train,y_train)
   not_nested_scores_knn [i] = clf_knn.best_score_

   # Nested CV with parameter optimization
   nested_score_knn = cross_val_score(clf_knn, x_train,y_train, cv=outer_cv)
   nested_scores_knn[i] = nested_score_knn.mean()
   print("nested_score",nested_score_knn)
print("R2 nested:",nested_scores_knn.mean() )
print("R2 not nested:",not_nested_scores_knn.mean())
print( clf_knn.best_params_)

In [ ]:
plt.style.use('seaborn')
plt.tight_layout()
plt.figure(figsize=(7,5))
sns.set(font_scale=1.2)
outer_scores_line, = plt.plot(not_nested_scores_knn, color='orange', linewidth=3.5)
nested_line, = plt.plot(nested_scores_knn, color='steelblue',  linewidth=3.5)
plt.ylabel("Cross-validation score", fontsize="15")
plt.xlabel("Number of trials", fontsize="15")
plt.legend([outer_scores_line, nested_line],
          ["Non-Nested CV_kNN", "Nested CV_kNN"],
          bbox_to_anchor=(0, .2, 1, 0), fontsize = 13)
# plt.title("Non-Nested vs Nested Cross-Validation",
#          x=.5, y=1.1, fontsize=15)
# plt.savefig('Cross-val_KNN.png', dpi = 1000, bbox_inches='tight')

# LASSO 

In [ ]:
grid_param_Lasso = {'estimator__alpha': [0,1], 'estimator__max_iter': [1000,1200,1500,200], 
                    'estimator__tol': [0.0001,0.001,0.01,0.1],
                   'estimator__fit_intercept': [True]
                    }
#Create arrays to store the scores
not_nested_scores_Lasso  = np.zeros(rounds)
nested_scores_Lasso = np.zeros(rounds)

for i in range(rounds):

   #Define both cross-validation objects (inner & outer)
   inner_cv = KFold(n_splits=5, shuffle=True, random_state=i)
   outer_cv = KFold(n_splits=3, shuffle=True, random_state=i)


   # Non-nested parameter search and scoring
   clf_Lasso  = GridSearchCV(estimator=MultiOutputRegressor(Lasso()), param_grid=grid_param_Lasso , cv=inner_cv)
   clf_Lasso.fit(x_train,y_train)
   not_nested_scores_Lasso [i] = clf_Lasso.best_score_

   # Nested CV with parameter optimization
   nested_score_Lasso = cross_val_score(clf_Lasso, x_train,y_train, cv=outer_cv)
   nested_scores_Lasso[i] = nested_score_Lasso.mean()
   print("nested_score",nested_score_Lasso )
print("R2 nested:",nested_scores_Lasso.mean() )
print("R2 not_nested:",not_nested_scores_Lasso.mean())
print( clf_Lasso.best_params_)

In [ ]:
plt.style.use('seaborn')
plt.tight_layout()
plt.figure(figsize=(7,5))
sns.set(font_scale=1.2)
outer_scores_line, = plt.plot(not_nested_scores_Lasso, color='orange',linewidth=3.5)
nested_line, = plt.plot(nested_scores_Lasso, color='steelblue', linewidth=3.5)
plt.ylabel("Cross-validation score", fontsize="15")
plt.xlabel("Number of trials", fontsize="15")
plt.ylim(0.85, 0.99)
plt.legend([outer_scores_line, nested_line],
          ["Non-Nested CV_Lasso", "Nested CV_Lasso"],
          bbox_to_anchor=(0, .2, 1, 0), fontsize = 13)
# plt.title("Non-Nested vs Nested Cross-Validation",
#          x=.5, y=1.1, fontsize="15")
# plt.savefig('Cross_val Lasso.png', dpi = 1000, bbox_inches='tight')

# MLP 

In [ ]:
grid_param_MLP = { 'estimator__activation':['identity', 'logistic', 'tanh', 'relu'], 
                  'estimator__alpha':[0.0001,0.0002], 'estimator__max_iter': [500, 1000,1500], 'estimator__learning_rate_init':
                 [0.001,0.002,0.015]}
#Create arrays to store the scores
not_nested_scores_MLP  = np.zeros(rounds)
nested_scores_MLP = np.zeros(rounds)

for i in range(rounds):

   #Define both cross-validation objects (inner & outer)
   inner_cv = KFold(n_splits=5, shuffle=True, random_state=i)
   outer_cv = KFold(n_splits=3, shuffle=True, random_state=i)


   # Non-nested parameter search and scoring
   clf_MLP = GridSearchCV(estimator=MultiOutputRegressor(MLPRegressor()), param_grid=grid_param_MLP , cv=inner_cv)
   clf_MLP.fit(x_train,y_train)
   not_nested_scores_MLP [i] = clf_MLP.best_score_

   # Nested CV with parameter optimization
   nested_score_MLP = cross_val_score(clf_MLP, x_train,y_train, cv=outer_cv)
   nested_scores_MLP[i] = nested_score_MLP.mean()
   print("nested_score",nested_score_MLP )
print("R2 nested:",nested_scores_MLP.mean() )
print("R2 non nested:",not_nested_scores_MLP.mean())
print( clf_MLP.best_params_)

In [ ]:
plt.style.use('seaborn')

plt.tight_layout()
plt.figure(figsize=(7,5))
sns.set(font_scale=1.2)
outer_scores_line, = plt.plot(not_nested_scores_MLP, color='orange',linewidth=3.5)
nested_line, = plt.plot(nested_scores_MLP, color='steelblue', linewidth=3.5)
plt.ylabel("Cross-validation score", fontsize="14")
plt.xlabel("Number of trials", fontsize="14")
plt.legend([outer_scores_line, nested_line],
          ["Non-Nested CV_MLP", "Nested CV_MLP"],
          bbox_to_anchor=(0, .2, .5, 0))
# plt.title("Non-Nested vs Nested Cross-Validation",
#          x=.5, y=1.1, fontsize="15")
# plt.savefig('Cross-val_MLP.png', dpi = 1000, bbox_inches='tight')

# Random Forest

In [ ]:
grid_param_RF = {'estimator__ccp_alpha':[0,0.01], 'estimator__max_depth':[1,2,3,4,5], 
                'estimator__n_estimators':[100,120,150,175,200,250]}
#Create arrays to store the scores
not_nested_scores_RF  = np.zeros(rounds)
nested_scores_RF = np.zeros(rounds)

for i in range(rounds):

   #Define both cross-validation objects (inner & outer)
   inner_cv = KFold(n_splits=5, shuffle=True, random_state=i)
   outer_cv = KFold(n_splits=3, shuffle=True, random_state=i)


   # Non-nested parameter search and scoring
   clf_RF  = GridSearchCV(estimator=MultiOutputRegressor(RandomForestRegressor()), param_grid=grid_param_RF , cv=inner_cv)
   clf_RF.fit(x_train,y_train)
   not_nested_scores_RF [i] = clf_RF.best_score_

   # Nested CV with parameter optimization
   nested_score_RF = cross_val_score(clf_RF, x_train,y_train, cv=outer_cv)
   nested_scores_RF[i] = nested_score_RF.mean()
   print("nested_score",nested_score_RF )
#    print("nested_score",not_nested_scores_RF )
print("R2 nested:",nested_scores_RF.mean() )
print("R2 non_nested:",not_nested_scores_RF.mean())
print( clf_RF.best_params_)

In [ ]:
plt.style.use('seaborn')
plt.tight_layout()
plt.figure(figsize=(7,5))
sns.set(font_scale=1.2)
outer_scores_line, = plt.plot(not_nested_scores_RF, color='orange', linewidth=3.5)
nested_line, = plt.plot(nested_scores_RF, color='steelblue', linewidth=3.5)
plt.ylabel("Cross-validation score", fontsize="15")
plt.xlabel("Number of trials", fontsize="15")
plt.legend([outer_scores_line, nested_line],
          ["Non-Nested CV_RF", "Nested CV_RF"],
          bbox_to_anchor=(0, .2, 1, 0), fontsize = 13)
# plt.title("Non-Nested vs Nested Cross-Validation", x=.5, y=1.1, fontsize="15")
# plt.savefig('Cross_val RandomFor.png', dpi = 1000, bbox_inches='tight')

# Ridge model 

In [ ]:
grid_param_Ridge = {'estimator__alpha': [0.01,0.1,0.2],        
                    'estimator__max_iter':[1000,1500,2000],                
                    'estimator__solver':['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga', 'lbfgs'],
                    'estimator__tol':[0.0001,0.001,0.1]
                    }
#Create arrays to store the scores
not_nested_scores_Ridge  = np.zeros(rounds)
nested_scores_Ridge = np.zeros(rounds)

for i in range(rounds):

   #Define both cross-validation objects (inner & outer)
   inner_cv = KFold(n_splits=5, shuffle=True, random_state=i)
   outer_cv = KFold(n_splits=3, shuffle=True, random_state=i)


   # Non-nested parameter search and scoring
   clf_Ridge = GridSearchCV(estimator=MultiOutputRegressor(Ridge()), param_grid=grid_param_Ridge , cv=inner_cv)
   clf_Ridge.fit(x_train,y_train)
   not_nested_scores_Ridge [i] = clf_Ridge.best_score_

   # Nested CV with parameter optimization
   nested_score_Ridge= cross_val_score(clf_Ridge, x_train,y_train, cv=outer_cv)
   nested_scores_Ridge[i] = nested_score_Ridge.mean()
   print("nested_score",nested_score_Ridge )
print("R2 nested:",nested_scores_Ridge.mean() )
print("R2 non nested:",not_nested_scores_Ridge.mean())
print( clf_Ridge.best_params_)

In [ ]:
plt.style.use('seaborn')
plt.tight_layout()
plt.figure(figsize=(7,5))
sns.set(font_scale=1.2)
outer_scores_line, = plt.plot(not_nested_scores_Ridge, color='orange', linewidth=3.5)
nested_line, = plt.plot(nested_scores_Ridge, color='steelblue', linewidth=3.5)
plt.ylabel("Cross-validation score", fontsize="15")
plt.xlabel("Number of trials", fontsize="15")
plt.legend([outer_scores_line, nested_line],
          ["Non-Nested CV_Ridge", "Nested CV_Ridge"],
          bbox_to_anchor=(0, .18, .5, 0), fontsize = 13)
# plt.title("Non-Nested vs Nested Cross-Validation",
#          x=.5, y=1.1, fontsize="15")
# plt.savefig('Cross_val Ridge.png', dpi = 1000, bbox_inches='tight')

# SVR Model 

In [ ]:
grid_param_SVR = {'estimator__C': [1,2,3], 'estimator__cache_size':[100,200], 'estimator__coef0':[0,0.01,0.001], 
                  'estimator__degree': [3,4,5], 'estimator__epsilon':[0.1,0.2,0.3], 'estimator__gamma':['auto'], 
                  'estimator__kernel':['linear', 'poly', 'rbf', 'sigmoid', 'precomputed']
                    }
#Create arrays to store the scores
not_nested_scores_SVR = np.zeros(rounds)
nested_scores_SVR = np.zeros(rounds)

for i in range(rounds):

   #Define both cross-validation objects (inner & outer)
   inner_cv = KFold(n_splits=5, shuffle=True, random_state=i)
   outer_cv = KFold(n_splits=3, shuffle=True, random_state=i)


   # Non-nested parameter search and scoring
   clf_SVR  = GridSearchCV(estimator=MultiOutputRegressor(SVR()), param_grid=grid_param_SVR , cv=inner_cv)
   clf_SVR.fit(x_train,y_train)
   not_nested_scores_SVR [i] = clf_SVR.best_score_

   # Nested CV with parameter optimization
   nested_score_SVR = cross_val_score(clf_SVR, x_train,y_train, cv=outer_cv)
   nested_scores_SVR[i] = nested_score_SVR.mean()
   print("nested_score",nested_score_SVR )
print("R2 nested:",nested_scores_SVR.mean() )
print("R2 non_nested:",not_nested_scores_SVR.mean())
print( clf_SVR.best_params_)

In [ ]:
plt.style.use('seaborn')
plt.tight_layout()
plt.figure(figsize=(7,5))
sns.set(font_scale=1.2)
outer_scores_line, = plt.plot(not_nested_scores_SVR, color='orange',linewidth=3.5)
nested_line, = plt.plot(nested_scores_SVR, color='steelblue', linewidth=3.5)
plt.ylabel("Cross-validation score", fontsize="15")
plt.xlabel("Number of trials", fontsize="15")
plt.legend([outer_scores_line, nested_line],
          ["Non-Nested CV_SVR", "Nested CV_SVR"],
          bbox_to_anchor=(0, .2, 1, 0), fontsize = 13)
# plt.title("Non-Nested vs Nested Cross-Validation",
#          x=.5, y=1.1, fontsize="15")
# plt.savefig('Cross_val SVR.png', dpi = 1000, bbox_inches='tight')

# XGB model 

In [ ]:
grid_param_XGB = {'estimator__alpha':[0.7,0.8,0.9,1.0,1.1], 'estimator__ccp_alpha':[0,0.001,0.01]}
#Create arrays to store the scores
not_nested_scores_XGB  = np.zeros(rounds)
nested_scores_XGB = np.zeros(rounds)

for i in range(rounds):

   #Define both cross-validation objects (inner & outer)
   inner_cv = KFold(n_splits=5, shuffle=True, random_state=i)
   outer_cv = KFold(n_splits=3, shuffle=True, random_state=i)


   # Non-nested parameter search and scoring
   clf_XGB = GridSearchCV(estimator=MultiOutputRegressor(GradientBoostingRegressor()), param_grid=grid_param_XGB , cv=inner_cv)
   clf_XGB.fit(x_train,y_train)
   not_nested_scores_XGB [i] = clf_XGB.best_score_

   # Nested CV with parameter optimization
   nested_score_XGB = cross_val_score(clf_XGB, x_train,y_train, cv=outer_cv)
   nested_scores_XGB[i] = nested_score_XGB.mean()
   print("nested_score",nested_score_XGB )
print("R2 nested:",nested_scores_XGB.mean() )
print("R2 non_nested:",not_nested_scores_XGB.mean())
print( clf_XGB.best_params_)

In [ ]:
plt.style.use('seaborn')
sns.set(font_scale=1.2)
plt.tight_layout()
plt.figure(figsize=(7,5))
outer_scores_line, = plt.plot(not_nested_scores_XGB, color='orange', linewidth=3.5)
nested_line, = plt.plot(nested_scores_XGB, color='steelblue', linewidth=3.5)
plt.ylabel("Cross-validation score", fontsize="15")
plt.xlabel("Number of trials", fontsize="15")
plt.legend([outer_scores_line, nested_line],
          ["Non-Nested CV_XGB", "Nested CV_XGB"],
          bbox_to_anchor=(0, .2, 1, 0),  fontsize="13")
# plt.title("Non-Nested vs Nested Cross-Validation",
#          x=.5, y=1.1, fontsize="15")
# plt.savefig('Cross_val XGB.png', dpi = 1000, bbox_inches='tight')